# Face Detection Pipeline

I am building a Face Detection Pipeline using the Open CV frontal face Haar cascade detection model (as we did in week 4). 

I will improve the output in two steps:
1. Remove multiple boxes for the same face using Non-Maximum Suppression (NMS).
2. Verify that each box actually contains a face with our trained HIOG feature classifier from week 5. 


I will measure the quality of the approaches with the WIDER FACE dataset.



## Ingest the WIDER FACE label data

In [ ]:
# The label data text file is structured as follows:
# - All image data are listed sequentially, with no blank lines in between.
# - The first line contains the relative path to the image.
# - The second line contains the number of faces in the image.
# - The subsequent lines contain the bounding box information for each face, formatted as "x y w h", where (x, y) is the top-left corner of the bounding box, and w and h are the width and height of the bounding box, respectively.
# - There are additional values on each box line, but we will ignore them for this task.
# - An image with 0 faces is followed by one row of 0 0 0 0 

images = []

path_to_file = "data/wider_face_split/wider_face_train_bbx_gt.txt"
with open(path_to_file, "r") as f:
    line = f.readline()
    # Stop when no more lines to read
    while line:
        # First line in an image block is the image path
        image_path = line.strip()
        # Second line is the number of faces in the image
        num_faces = int(f.readline().strip())
        faces = []
        # store the bounding box information for each face in the image
        # note that if the number of faces is 0, this block is not executed
        for _ in range(num_faces):
            face_info = f.readline().strip().split()
            x, y, w, h = map(int, face_info[:4])
            faces.append((x, y, w, h))
        # Handling the case of 0 faces: Ignore the next line which contains "0 0 0 0"
        if num_faces == 0:
            line = f.readline()
        # Add image data to the list
        images.append((image_path, faces))
        # Read the next image path for the next iteration (or to end the loop)
        line = f.readline()

# the format of each entry is (image_path, [(x1, y1, w1, h1), (x2, y2, w2, h2), ...])
# hence, number of faces in the image can be obtained by len(faces) for each entry, e.g., len(images[0][1]) gives the number of faces in the first image        
# print the first 5 images and their faces for validation
for image in images[:5]:
    print(image)


('0--Parade/0_Parade_marchingband_1_849.jpg', [(449, 330, 122, 149)])
('0--Parade/0_Parade_Parade_0_904.jpg', [(361, 98, 263, 339)])
('0--Parade/0_Parade_marchingband_1_799.jpg', [(78, 221, 7, 8), (78, 238, 14, 17), (113, 212, 11, 15), (134, 260, 15, 15), (163, 250, 14, 17), (201, 218, 10, 12), (182, 266, 15, 17), (245, 279, 18, 15), (304, 265, 16, 17), (328, 295, 16, 20), (389, 281, 17, 19), (406, 293, 21, 21), (436, 290, 22, 17), (522, 328, 21, 18), (643, 320, 23, 22), (653, 224, 17, 25), (793, 337, 23, 30), (535, 311, 16, 17), (29, 220, 11, 15), (3, 232, 11, 15), (20, 215, 12, 16)])
('0--Parade/0_Parade_marchingband_1_117.jpg', [(69, 359, 50, 36), (227, 382, 56, 43), (296, 305, 44, 26), (353, 280, 40, 36), (885, 377, 63, 41), (819, 391, 34, 43), (727, 342, 37, 31), (598, 246, 33, 29), (740, 308, 45, 33)])
('0--Parade/0_Parade_marchingband_1_778.jpg', [(27, 226, 33, 36), (63, 95, 16, 19), (64, 63, 17, 18), (88, 13, 16, 15), (231, 1, 13, 13), (263, 122, 14, 20), (367, 68, 15, 23), (19

## IoU calculator

In order to identify whether two boxes refer to the same object (face), I calculate the "Intersection over Union" measure which calculates the overlap to total (union) size ratio. 

The measure calculates the intersection of two boxes, as well as the area that is covered by both boxes together (i.e. area of both boxes minus the intersection area to not count that twice.)

If two boxes are identical IoU is 1 (intersection = union); if they do not overlap, IoU is 0 (intersection is 0).

As different models / labelling may apply different padding to the faces, and boxes may be moved by some pixels, exact box matching cannot be expected. A standard measure to decide that two boxes box the same object is IoU > 0.5 which I will apply here. However, that value is to a degree arbitrary.

In [11]:
def calculate_IoU(boxA, boxB):
    # boxA and boxB are in the format (x, y, w, h)

    # first calculate the (x, y) coordinates of the intersection rectangle
    i_x_top_left = max(boxA[0], boxB[0])
    i_y_top_left = max(boxA[1], boxB[1])
    i_x_bottom_right = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    i_y_bottom_right = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    # Compute the area of intersection rectangle
    # If the rectangles do not overlap, i_x_bottom_right - i_x_top_left 
    # or i_y_bottom_right - i_y_top_left will be negative, 
    # so we take max with 0 to ensure non-negative area
    i_width = max(0, i_x_bottom_right - i_x_top_left)
    i_height = max(0, i_y_bottom_right - i_y_top_left)
    # If no overlap, the intersection area will be 0
    i_area = i_width * i_height

    # Compute the area of both boxes based on width*height
    boxA_area = boxA[2] * boxA[3]
    boxB_area = boxB[2] * boxB[3]

    # Compute the intersection over union by taking the intersection area 
    # and dividing it by the sum of prediction + ground-truth areas - the interesection area
    iou = i_area / float(boxA_area + boxB_area - i_area)

    return iou  

In [12]:
print(calculate_IoU((10, 10, 50, 50), (30, 30, 50, 50)))

0.21951219512195122
